In [ ]:
import json
import csv
import sqlite3
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import re

def FindEducationTage(edu):
    if edu == "High School":
        edutag = 'high school'
    elif edu == 'Middle School':
        edutag = 'middle school'
    elif edu == 'Bachelors Degree':
        edutag = 'bachelor'
    elif edu == 'Masters Degree':
        edutag = 'master'
    elif edu == 'Post-Graduate Degree':
        edutag = 'pg'
    else:
        raise Exception("Education Tag undefined: " + edu)
        
    return edutag


def FindGenderTage(gender):
    if gender == "Male":
        gendertag = 'M'
    elif gender == 'Female':
        gendertag = 'F'    
    else:
        raise Exception("Gender Tag undefined: " + gender)
    return gendertag


def FindHitLen(convname):
    hitid = int(convname[re.search(r'static/dataset/conv', convname).end():re.search(r'_len', convname).start()])
    convlen = int(convname[re.search(r'_len', convname).end():re.search(r'.html', convname).start()])
    return hitid, convlen


def FindTopicChoice(optionlist, topicsel):
    count = 0
    for to in optionlist:
        if to == topicsel:
            return count+1
        else:
            count = count + 1
    if count == 0:
        raise Exception("TopicChoice undefined")
        
        
def FindTuringResTage(response):
    if response == 'A= Human, B = Human':
        restag = 'HH'
    elif response == 'A= Human, B = Machine':
        restag = 'HAI'
    elif response == 'A= Machine, B = Human':
        restag = 'AIH'
    elif response == 'A= Machine, B = Machine':
        restag = 'AIAI'
    else:
        raise Exception("TuringRes Tag undefined")
        
    return restag

In [ ]:
## sorted the data based on the following criteria in the journal:
##\MM{other tasks: main text: combine + supp: in-lab} 
##\MM{conversation: main text: in lab + supp: mturk + prolific}

results = {'inlab':{'datastring':[]}, 'prolific':{'datastring':[]}, 'mturk':{'datastring':[]}, 'chatgpt':{'datastring':[]}}


############# for jason file processing
jsonpath = './Data_all/'
filenames = {'inlab': ['Marcelo_InLab_With_Gender_3','Prachi_InLab_With_Gender_2','Prachi_InLab_Without_Gender_2',
                      'Shui_InLab_With_Gender_16','Shui_InLab_Without_Gender_17', 'Tanish_InLab_WithGender'],
             'prolific':['prolific_withgender50participants','prolific_withoutgender52participants']}

for key in filenames:
    print("=======================")
    print(key)

    for jasonfilename in filenames[key]:
        #print("=======================")
        #print(jasonfilename)
        f = open(jsonpath + jasonfilename + '.json')
        data = json.load(f)
        #print(len(data))
        for workerid in range(len(data)):
            #print(data[workerid])
            demographic_trialid = 0
            while not ('rawResponses' in data[workerid][demographic_trialid]):
                demographic_trialid = demographic_trialid + 1
            
            workerdata = {'questiondata':{'gender':0,'age':0,'native':0,'education':0,'country':0},
                          'data':[{'trialdata':{'hit':0,'topic':0,'hit_len':0,'Aclass':0,'rt':0,
                                                'Agender': 0, 'Bgender': 0, 'conversationID': '.html'}}]}

            workerdata['questiondata']['gender'] = data[workerid][demographic_trialid]['rawResponses']['A1'].lower()
            workerdata['questiondata']['country'] = data[workerid][demographic_trialid]['rawResponses']['A3']
            workerdata['questiondata']['age'] = data[workerid][demographic_trialid]['rawResponses']['A0'][:-10]
            workerdata['questiondata']['native'] = "yes"
            workerdata['questiondata']['education'] = FindEducationTage(data[workerid][demographic_trialid]['rawResponses']['A2'])
            
            official_trialid = demographic_trialid + 1
            while not ('conversation_name1' in data[workerid][official_trialid]):
                official_trialid = official_trialid + 1
            
            for trialid in range(official_trialid,len(data[workerid])-2): #starting from 5!!! the first few are not valid trials
                trialdata = {'trialdata':{'hit':0,'topic':0,'hit_len':0,'Aclass':0,'rt':0,
                                                'Agender': 0, 'Bgender': 0, 'conversationID': '.html'}}

                #print("=========================")
                #print(data[workerid][trialid])
                trialdata['trialdata']['rt'] = data[workerid][trialid]['rt']
                convname = data[workerid][trialid]['conversation_name1']
                trialdata['trialdata']['conversationID'] = convname
                hitid, convlen = FindHitLen(convname)
                trialdata['trialdata']['hit'] = hitid
                trialdata['trialdata']['hit_len'] = convlen
                optionlist = [data[workerid][trialid]['optionslistone'], 
                             data[workerid][trialid]['optionslisttwo'],
                             data[workerid][trialid]['optionslistthree'],
                             data[workerid][trialid]['optionslistfour'],
                             data[workerid][trialid]['optionslistfive']]
                topicsel = data[workerid][trialid]['response']['topic']
                topic = FindTopicChoice(optionlist, topicsel)
                trialdata['trialdata']['topic'] = str(topic)
                
                if 'gender1' in data[workerid][trialid]['response']:
                    trialdata['trialdata']['Agender'] = FindGenderTage(data[workerid][trialid]['response']['gender1'])
                    trialdata['trialdata']['Bgender'] = FindGenderTage(data[workerid][trialid]['response']['gender2'])
                
                trialdata['trialdata']['Aclass'] = FindTuringResTage(data[workerid][trialid]['response']['type1'])
                #print("=========================")
                print(trialdata)
                workerdata['data'].append(trialdata)

            #print("=========================")
            #print(workerdata)
            results[key]['datastring'].append(workerdata)
            
print("=========================")
print("number of in-lab pariticpants:")
print(len(results['inlab']['datastring']))
print("number of prolific pariticpants:")
print(len(results['prolific']['datastring']))


What Elisa used for her Mturk with feedback 

In [1]:
import json
import csv
import sqlite3
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:

results = {'prolific':{'datastring':[]}}

# Connect to your database
db_path = r"/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/Plot/Data_all/participants_prolific_may30.db"
con = sqlite3.connect(db_path)

# List all tables
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables inside the database:", tables)


Tables inside the database: [('apscheduler_jobs',), ('TuringConversationalAI',), ('amt_hit',), ('campaign',)]


In [ ]:
############# for db file processing 
file_path = db_path #r"/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/conversation_task/participants_AIexperts.db"
con = sqlite3.connect(file_path)
# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from TuringConversationalAI", con)
# = sqlite3.connect('./Data_all/Giorgia_all_inlab_online_2.db')
# Load the data into a DataFrame
#df2 = pd.read_sql_query("SELECT * from TuringConversationalAI", con)
#df = pd.concat([df1,df2], ignore_index=True)

num_conv = 32
key = 'prolific'
for w in range(len(df)):
    if df["mode"][w] in ["debug"] and df["status"][w] == 3 and df["datastring"][w] is not None:
        #data = json.loads(df.loc[w].at["datastring"])
        
        workerdata = {'questiondata':{},
                          'data':[]}
        
        data = json.loads(df["datastring"][w])
        Wgender = data['questiondata']['gender']
        Wage = data['questiondata']['age']
        Wlanguage = data['questiondata']['native']
        Wdegree = data['questiondata']['education']
        Wfield = data['questiondata']['country']        
        
        workerdata['questiondata']['gender'] = Wgender
        workerdata['questiondata']['age'] = Wage
        workerdata['questiondata']['native'] = Wlanguage
        workerdata['questiondata']['education'] = Wdegree
        workerdata['questiondata']['country'] = Wfield       
        
        for c in range(num_conv):
            
            trialdata = {'trialdata':{'hit':0,'topic':0,'hit_len':0,'Aclass':0,'rt':0,
                                                'Agender': 0, 'Bgender': 0, 'conversationID': '.html'}}
            
            #print(data['data'][1+c]['trialdata'])
            # load responses to conv1
            conv1_hit = data['data'][2+c]['trialdata']['hit']
            conv1_len = data['data'][2+c]['trialdata']['hit_len']
            conv1_combclass = data['data'][2+c]['trialdata']['Aclass']
            conv1_topic = data['data'][2+c]['trialdata']['topic']
            conv1_convname = 'static/dataset/conv'+str(conv1_hit)+'_len'+str(conv1_len)+'.html'
            conv1_rt = data['data'][2+c]['trialdata']['rt'] # absolute
            conv1_Agender = data['data'][2+c]['trialdata']['Agender']
            conv1_Bgender = data['data'][2+c]['trialdata']['Bgender']
            is_catch = data['data'][2+c]['trialdata']['isCatch']
            conv1_hasFeedback = data['data'][2+c]['trialdata']['hasFeedback']
            
            trialdata['trialdata']['rt'] = conv1_rt
            trialdata['trialdata']['hit'] = conv1_hit
            trialdata['trialdata']['topic'] = conv1_topic
            trialdata['trialdata']['hit_len'] = conv1_len
            trialdata['trialdata']['Aclass'] = conv1_combclass
            trialdata['trialdata']['Agender'] = conv1_Agender
            trialdata['trialdata']['Bgender'] = conv1_Bgender
            trialdata['trialdata']['conversationID'] = conv1_convname
            trialdata['trialdata']['isCatch'] = is_catch
            trialdata['trialdata']['hasFeedback'] = conv1_hasFeedback
            
            workerdata['data'].append(trialdata)
        results[key]['datastring'].append(workerdata)
        
print("=========================")
print("number of prolific pariticpants:")
print(len(results['prolific']['datastring']))


KeyError: 'AIexperts'

In [ ]:
#save the compiled results as jason
with open("./savedResults_elisa/compiled_prolific.json", "w") as fp:
    json.dump(results,fp) 
    
print('Compilation completed! Results are saved to folder!')

# Preprocess for confusion matrices divided by topic

In [1]:
import pandas as pd
import json

def load_topic_mapping(excel_path, sheet_name=None):
    """
    Load the Excel sheet and create a dictionary mapping conversation_id to topic
    """
    df = pd.read_excel(excel_path, sheet_name='Sheet1')
    topic_mapping = {}
    for _, row in df.iterrows():
        conv_id = str(int(row.iloc[0]))  # First column (conversation_id)
        topic = row.iloc[33] if len(row) > 33 else None  # Column AH (34th column, 0-indexed = 33)
        if pd.notna(topic):
            topic_mapping[conv_id] = topic.lower().strip()
    return topic_mapping

def reorganize_by_topic(compiled_json_path, excel_path, output_path="/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/Plot/savedResults_elisa/compiled_by_topic.json"):
    """
    Reorganize existing compiled JSON by topic using Excel mapping
    """
    # Load topic mapping
    topic_mapping = load_topic_mapping(excel_path)
    print(f"Loaded {len(topic_mapping)} conversation-topic mappings")
    
    # Load existing compiled JSON
    with open(compiled_json_path, 'r') as f:
        original_data = json.load(f)
    
    # Initialize results structure by topic - preserve original source structure
    topics = ['science', 'technology', 'books', 'movies', 'music', 'politics', 
              'general entertainment', 'fashion', 'sports', 'food']
    
    # Get source keys from original data (inlab, prolific, etc.)
    source_keys = [key for key in original_data.keys() if 'datastring' in original_data.get(key, {})]
    
    results_by_topic = {}
    for topic in topics:
        results_by_topic[topic] = {}
        for source_key in source_keys:
            results_by_topic[topic][source_key] = {'datastring': []}
    
    # Also keep original structure for reference
    results_by_topic['all'] = original_data
    
    # Process each data source (inlab, prolific, etc.)
    for source_key in source_keys:
        print(f"\nProcessing {source_key} data...")
        
        for participant in original_data[source_key]['datastring']:
            # Group this participant's trials by topic
            trials_by_topic = {}
            
            for trial in participant['data']:
                if 'trialdata' not in trial:
                    continue
                    
                # Get conversation ID (hit number)
                hit_id = str(trial['trialdata']['hit'])
                
                # Look up topic from mapping
                assigned_topic = topic_mapping.get(hit_id, 'nolabel')
                
                # Add assigned_topic to trial data
                trial['trialdata']['assigned_topic'] = assigned_topic
                
                # Group by topic
                if assigned_topic not in trials_by_topic:
                    trials_by_topic[assigned_topic] = []
                trials_by_topic[assigned_topic].append(trial)
            
            # Create separate participant entries for each topic they have trials for
            for topic, topic_trials in trials_by_topic.items():
                if topic_trials and topic in results_by_topic:
                    participant_topic_data = {
                        'questiondata': participant['questiondata'].copy(),
                        'data': topic_trials
                    }
                    results_by_topic[topic][source_key]['datastring'].append(participant_topic_data)
    
    # Save reorganized data
    with open(output_path, 'w') as f:
        json.dump(results_by_topic, f, indent=2)
    
    # Print summary statistics
    print(f"\n=========================")
    print("REORGANIZATION COMPLETE")
    print(f"Output saved to: {output_path}")
    print("=========================")
    print("SUMMARY BY TOPIC:")
    print("=========================")
    
    for topic in topics:
        total_count = sum(len(results_by_topic[topic][source]['datastring']) 
                         for source in results_by_topic[topic])
        if total_count > 0:
            breakdown = []
            for source in results_by_topic[topic]:
                count = len(results_by_topic[topic][source]['datastring'])
                if count > 0:
                    breakdown.append(f"{source}: {count}")
            breakdown_str = ", ".join(breakdown)
            print(f"{topic.upper()}: {total_count} participants ({breakdown_str})")
    
    total_original = sum(len(original_data[source]['datastring']) 
                        for source in source_keys)
    print(f"\nOriginal total participants: {total_original}")
    
    return results_by_topic, topic_mapping

def save_individual_topic_files(results_by_topic, output_dir="/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/Plot/savedResults_elisa/topic_data/"):
    """
    Save individual JSON files for each topic in original format
    """
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    topics = ['science', 'technology', 'books', 'movies', 'music', 'politics', 
              'general entertainment', 'fashion', 'sports', 'food']
    
    for topic in topics:
        # Check if topic has any data
        has_data = any(len(results_by_topic[topic][source]['datastring']) > 0 
                      for source in results_by_topic[topic])
        
        if has_data:
            # Save in exact original format: {"inlab": {"datastring": [...]}, "prolific": {"datastring": [...]}}
            topic_data = results_by_topic[topic]
            filename = f"{output_dir}compiled_{topic}.json"
            
            with open(filename, 'w') as f:
                json.dump(topic_data, f, indent=2)
            
            # Count participants for this topic
            total_participants = sum(len(topic_data[source]['datastring']) 
                                   for source in topic_data)
            print(f"Saved {topic} data to {filename} ({total_participants} participants)")

def main(compiled_json_path, excel_path, output_path="/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/Plot/savedResults_elisa/compiled_by_topic.json"):
    """
    Main function to reorganize compiled JSON by topics
    
    Args:
        compiled_json_path: Path to your existing compiled.json file
        excel_path: Path to Excel file containing conversation_id and topic mapping
        output_path: Path for output file (default: compiled_by_topic.json)
    """
    try:
        results_by_topic, topic_mapping = reorganize_by_topic(
            compiled_json_path, excel_path, output_path
        )
        
        # Optionally save individual topic files
        save_individual_topic_files(results_by_topic)
        
        print(f"\nSuccess! Found topics: {list(set(topic_mapping.values()))}")
        return results_by_topic, topic_mapping
        
    except Exception as e:
        print(f"Error processing data: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Example usage:
# results, mapping = main(
#     compiled_json_path="your_compiled.json",
#     excel_path="your_topic_mapping.xlsx"
# )

# Quick function to inspect what's in your compiled JSON
def inspect_compiled_json(json_path):
    """
    Quick inspection of compiled JSON structure
    """
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    print("JSON Structure:")
    for key in data:
        if isinstance(data[key], dict) and 'datastring' in data[key]:
            participant_count = len(data[key]['datastring'])
            print(f"  {key}: {participant_count} participants")
            
            # Look at first participant to see trial structure
            if participant_count > 0:
                first_participant = data[key]['datastring'][0]
                trial_count = len(first_participant['data'])
                print(f"    - First participant has {trial_count} trials")
                
                # Show a sample trial
                if trial_count > 1:  # Skip the first dummy trial
                    sample_trial = first_participant['data'][1]['trialdata']
                    print(f"    - Sample trial hit: {sample_trial.get('hit', 'N/A')}")
                    print(f"    - Sample trial topic: {sample_trial.get('topic', 'N/A')}")


In [3]:
json_path = "/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/Plot/savedResults/compiled.json"
inspect_compiled_json(json_path)

JSON Structure:
  inlab: 50 participants
    - First participant has 21 trials
    - Sample trial hit: 22
    - Sample trial topic: 1
  prolific: 102 participants
    - First participant has 21 trials
    - Sample trial hit: 207
    - Sample trial topic: 5
  mturk: 200 participants
    - First participant has 21 trials
    - Sample trial hit: 33
    - Sample trial topic: 2
  chatgpt: 15 participants
    - First participant has 21 trials
    - Sample trial hit: 3
    - Sample trial topic: games


In [ ]:
output_path="/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/Plot/savedResults_elisa/compiled_by_topic.json"

In [4]:
results, topic_mapping = main(
    compiled_json_path= json_path,
    excel_path="/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/conversation_task_for_revisions/code_dataanalsysis/record_conversations_paper.xlsx"
)

Loaded 299 conversation-topic mappings

Processing inlab data...

Processing prolific data...

Processing mturk data...

Processing chatgpt data...

REORGANIZATION COMPLETE
Output saved to: /Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/Plot/savedResults_elisa/compiled_by_topic.json
SUMMARY BY TOPIC:
SCIENCE: 304 participants (inlab: 42, prolific: 82, mturk: 165, chatgpt: 15)
TECHNOLOGY: 270 participants (inlab: 42, prolific: 77, mturk: 151)
BOOKS: 228 participants (inlab: 32, prolific: 62, mturk: 119, chatgpt: 15)
MOVIES: 356 participants (inlab: 49, prolific: 99, mturk: 193, chatgpt: 15)
MUSIC: 347 participants (inlab: 47, prolific: 97, mturk: 188, chatgpt: 15)
POLITICS: 244 participants (inlab: 38, prolific: 69, mturk: 137)
GENERAL ENTERTAINMENT: 350 participants (inlab: 48, prolific: 97, mturk: 190, chatgpt: 15)
FASHION: 157 participants (inlab: 23, prolific: 42, mturk: 77, chatgpt: 15)
SPORTS: 328 participants (inlab: 46, prol

In [ ]:
############# for db file processing (in-lab)
con = sqlite3.connect('./Data_all/Giorgia_all_inlab_online_1.db')
# Load the data into a DataFrame
df1 = pd.read_sql_query("SELECT * from TuringConversationalAI", con)
con = sqlite3.connect('./Data_all/Giorgia_all_inlab_online_2.db')
# Load the data into a DataFrame
df2 = pd.read_sql_query("SELECT * from TuringConversationalAI", con)
df = pd.concat([df1,df2], ignore_index=True)

num_conv = 20
key = 'mturk'
for w in range(len(df)):
    if df["mode"][w] in ["live"] and df["endhit"][w] is not None:
        #data = json.loads(df.loc[w].at["datastring"])
        
        workerdata = {'questiondata':{'gender':0,'age':0,'native':0,'education':0,'country':0},
                          'data':[{'trialdata':{'hit':0,'topic':0,'hit_len':0,'Aclass':0,'rt':0,
                                                'Agender': 0, 'Bgender': 0, 'conversationID': '.html'}}]}
        
        data = json.loads(df["datastring"][w])
        Wgender = data['questiondata']['gender']
        Wage = data['questiondata']['age']
        Wlanguage = data['questiondata']['native']
        Wdegree = data['questiondata']['education']
        Wfield = data['questiondata']['country']        
        
        workerdata['questiondata']['gender'] = Wgender
        workerdata['questiondata']['age'] = Wage
        workerdata['questiondata']['native'] = Wlanguage
        workerdata['questiondata']['education'] = Wdegree
        workerdata['questiondata']['country'] = Wfield       
        
        for c in range(num_conv):
            
            trialdata = {'trialdata':{'hit':0,'topic':0,'hit_len':0,'Aclass':0,'rt':0,
                                                'Agender': 0, 'Bgender': 0, 'conversationID': '.html'}}
            
            #print(data['data'][1+c]['trialdata'])
            # load responses to conv1
            conv1_hit = data['data'][1+c]['trialdata']['hit']
            conv1_len = data['data'][1+c]['trialdata']['hit_len']
            conv1_combclass = data['data'][1+c]['trialdata']['Aclass']
            conv1_topic = data['data'][1+c]['trialdata']['topic']
            conv1_convname = 'static/dataset/conv'+str(conv1_hit)+'_len'+str(conv1_len)+'.html'
            conv1_rt = data['data'][1+c]['trialdata']['rt'] # absolute
            conv1_Agender = data['data'][1+c]['trialdata']['Agender']
            conv1_Bgender = data['data'][1+c]['trialdata']['Bgender']
            
            trialdata['trialdata']['rt'] = conv1_rt
            trialdata['trialdata']['hit'] = conv1_hit
            trialdata['trialdata']['topic'] = conv1_topic
            trialdata['trialdata']['hit_len'] = conv1_len
            trialdata['trialdata']['Aclass'] = conv1_combclass
            trialdata['trialdata']['Agender'] = conv1_Agender
            trialdata['trialdata']['Bgender'] = conv1_Bgender
            trialdata['trialdata']['conversationID'] = conv1_convname
            
            workerdata['data'].append(trialdata)
        results[key]['datastring'].append(workerdata)
        


In [ ]:
############# for db file processing (chatGPT)
con = sqlite3.connect('./Data_all/Mranmay_inlab_ChatGPTOnlyWoGenders.db')
# Load the data into a DataFrame
df1 = pd.read_sql_query("SELECT * from TuringConversationalAI", con)
df = pd.concat([df1], ignore_index=True)

num_conv = 20
key = 'chatgpt'
for w in range(len(df)):
    
    if df["mode"][w] in ["debug"] and df["endhit"][w] is not None:
        #data = json.loads(df.loc[w].at["datastring"])
               
        workerdata = {'questiondata':{'gender':0,'age':0,'native':0,'education':0,'country':0},
                          'data':[{'trialdata':{'hit':0,'topic':0,'hit_len':0,'Aclass':0,'rt':0,
                                                 'conversationID': '.html'}}]}
        
        data = json.loads(df["datastring"][w])
        
        #print(data)
        
        Wgender = data['questiondata']['gender']
        Wage = data['questiondata']['age']
        Wlanguage = data['questiondata']['native']
        Wdegree = data['questiondata']['education']
        Wfield = data['questiondata']['country']        
        
        workerdata['questiondata']['gender'] = Wgender
        workerdata['questiondata']['age'] = Wage
        workerdata['questiondata']['native'] = Wlanguage
        workerdata['questiondata']['education'] = Wdegree
        workerdata['questiondata']['country'] = Wfield       
        
        for c in range(num_conv):
            #print(c)
            trialdata = {'trialdata':{'hit':0,'topic':0,'hit_len':0,'Aclass':0,'rt':0,
                                                'Agender': 0, 'Bgender': 0, 'conversationID': '.html'}}
            
            #print(data['data'][1+c]['trialdata'])
            # load responses to conv1
            #print(data['data'][2])
            
            conv1_hit = data['data'][1+c]['trialdata']['hit']
            conv1_len = data['data'][1+c]['trialdata']['hit_len']
            conv1_combclass = data['data'][1+c]['trialdata']['Aclass']
            conv1_topic = data['data'][1+c]['trialdata']['topic']
            conv1_convname = 'Mranmey_chatgpt_dataset_woGender/conv'+str(conv1_hit)+'_len'+str(conv1_len)+'.html'
            conv1_rt = data['data'][1+c]['trialdata']['rt'] # absolute
            #conv1_Agender = data['data'][1+c]['trialdata']['Agender']
            #conv1_Bgender = data['data'][1+c]['trialdata']['Bgender']
            
            trialdata['trialdata']['rt'] = conv1_rt
            trialdata['trialdata']['hit'] = conv1_hit
            trialdata['trialdata']['topic'] = conv1_topic
            trialdata['trialdata']['hit_len'] = conv1_len
            trialdata['trialdata']['Aclass'] = conv1_combclass
            #trialdata['trialdata']['Agender'] = conv1_Agender
            #trialdata['trialdata']['Bgender'] = conv1_Bgender
            trialdata['trialdata']['conversationID'] = conv1_convname
            
            workerdata['data'].append(trialdata)
        results[key]['datastring'].append(workerdata)
        
print("=========================")
print("number of in-lab pariticpants:")
print(len(results['inlab']['datastring']))
countTrial = 0
for work in range(len(results['inlab']['datastring'])):
    countTrial = countTrial + len(results['inlab']['datastring'][work]['data'])
print(countTrial)

print("number of prolific pariticpants:")
print(len(results['prolific']['datastring']))
countTrial = 0
for work in range(len(results['prolific']['datastring'])):
    countTrial = countTrial + len(results['prolific']['datastring'][work]['data'])
print(countTrial)

print("number of mturk pariticpants:")
print(len(results['mturk']['datastring']))
countTrial = 0
for work in range(len(results['mturk']['datastring'])):
    countTrial = countTrial + len(results['mturk']['datastring'][work]['data'])
print(countTrial)

print("number of chatgpt pariticpants:")
print(len(results['chatgpt']['datastring']))
countTrial = 0
for work in range(len(results['chatgpt']['datastring'])):
    countTrial = countTrial + len(results['chatgpt']['datastring'][work]['data'])
print(countTrial)

In [ ]:
#save the compiled results as jason
with open("./savedResults_elisa/compiled_prolific.json", "w") as fp:
    json.dump(results,fp) 
    
print('Compilation completed! Results are saved to folder!')

Compilation completed! Results are saved to folder!


In [ ]:
1050 + 2142 + 4200 + 315